In [ ]:
import os
import sys
import cv2
import pygmsh
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.patches import Rectangle, Polygon
import keyboard

# import tkinter as tk

In [ ]:
def transformation(old_points, new_points, data):
    old_points = old_points.reshape(-1, 1, 2)
    new_points = new_points.reshape(-1, 1, 2)
    data = data.reshape(-1, 1, 2)

    M, _ = cv2.findHomography(old_points, new_points, cv2.RANSAC, 5.0)
    transformed_data = cv2.perspectiveTransform(data, M)

    return transformed_data

In [ ]:
def circle_intersection(data):
    n = data.shape[0]

    # Vytvořte matici koeficientů soustavy rovnic
    A = np.zeros((n - 1, 2))
    for i in range(1, n):
        x_i, y_i = data[i, 0], data[i, 1]
        x_1, y_1 = data[0, 0], data[0, 1]
        A[i - 1] = 2 * (x_i - x_1), 2 * (y_i - y_1)

    # Vytvořte vektor pravých stran soustavy rovnic
    b = np.zeros(n - 1)
    x_1, y_1 = data[0, 0], data[0, 1]
    r_1 = data[0, 2]
    for i in range(1, n):
        x_i, y_i = data[i, 0], data[i, 1]
        r_i = data[i, 2]
        b[i - 1] = (x_i ** 2 + y_i ** 2 - r_i ** 2) - (x_1 ** 2 + y_1 ** 2 - r_1 ** 2)

    # Vypočtěte souřadnice průsečíku
    intersection = np.linalg.lstsq(A, b, rcond=None)[0]

    return intersection

In [ ]:
def second_circle_intersection(data, initial_guess):
    known_points = data[:, :2]
    distances = data[:, 2]
    from scipy.optimize import least_squares
    result = least_squares(distance_error, initial_guess, args=(distances, known_points))
    return result.x

In [ ]:
def show_results_graph(image_number):
    if image_number > len(image_files):
        image_number = len(image_files)

    index = image_files.index(image_files[image_number])

    print("\n\nVykreslení výsledků: Fotografie - {}: {}".format(index + 1, image_files[index]))

    fig, ax = plt.subplots(figsize=(int(0.0017 * width), int(0.0017 * height)))
    plt.title('Triangle elements - Image {}: {}'.format(index + 1, image_files[index]))

    gray_image = cv2.imread(os.path.join(main_image_folder, image_files[index]), 0)
    plt.imshow(gray_image, cmap='gray')

    next_data = triangle_vertices_all[index]
    new_center = triangle_centers_all[index]
    tri_index = triangle_indexes_all[index]
    upper_area_cor = correlation_area_points_all[index]
    triangle_points = triangle_points_all[index]
    triangle_coordinates = triangle_points[wrong_points_indexes_all[index]]

    patches = [Polygon(triangle, closed=True, facecolor='darkorange', alpha=0.45) for triangle in
               triangle_coordinates]
    ax.patches.extend(patches)

    plt.triplot(next_data[:, 0], next_data[:, 1], tri_index, color='green')

    plt.scatter(new_center[:, 0], new_center[:, 1],
                s=8,  # Velikost bodů
                marker='o')  # Znak bodů

    # Vykreslení obdélníka
    rec_w, rec_h = upper_area_cor[1, 0] - upper_area_cor[0, 0], upper_area_cor[1, 1] - upper_area_cor[0, 1]
    rect_patch = Rectangle((upper_area_cor[0]), rec_w, rec_h, edgecolor='firebrick', linewidth=1.5, facecolor='none')
    ax.add_patch(rect_patch)
    rect_patch = Rectangle((upper_area_cor[0]), rec_w, rec_h, facecolor='red', alpha=0.1)
    ax.add_patch(rect_patch)

    plt.axis('equal')
    plt.tight_layout()
    plt.show()

In [ ]:
def distance_error(point, distances, known_points):
    x_i, y_i = point
    known_points = np.array(known_points)
    distances = np.array(distances)

    errors = (np.sqrt((x_i - known_points[:, 0]) ** 2 + (y_i - known_points[:, 1]) ** 2) - distances) ** 2
    error = np.sum(errors)
    return error

In [ ]:
def point_track(mesh, current_shift=None, m1=None, m2=None):
    global keypoints1, descriptors1

    method = cv2.SIFT_create(
        nfeatures=0,  # __________________ Počet detekovaných rysů (0 = všechny dostupné) ______ def = 0
        nOctaveLayers=3,  # ______________ Počet vrstev v každé oktávě _________________________ def = 3
        contrastThreshold=0.04,  # _______ Práh kontrastu pro platnost rysu ____________________ def = 0.04
        edgeThreshold=10,  # _____________ Práh hrany pro platnost rysu blízko k okraji ________ def = 10
        sigma=1.6  # _____________________ Gaussovská hladina oktáv ____________________________ def = 1.6
    )

    x_old, y_old, x_new, y_new, angle_old, angle_new, index = [], [], [], [], [], [], [0]
    counter = 0

    length = len(mesh.get_cells_type("triangle"))

    print("\n\tVytváření hledaných bodů.")

    if isinstance(m1, np.ndarray) and isinstance(m2, np.ndarray):
        mask1 = m1
        mask2 = m2
    else:
        mask1 = np.zeros(gray1.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask1, [points1], 255)
        try:
            cv2.fillPoly(mask1, [points2], 0)
        except cv2.error:
            pass

        mask2 = np.zeros(gray2.shape[:2], dtype=np.uint8)
        cv2.rectangle(mask2, (1400, int(1050 + current_shift)), (4600, 3200), 255, -1)
        # cv2.rectangle(mask2, (ROI_area[0] - 50, ROI_area[1] + 50), (ROI_area[2] - 50, ROI_area[3] + 50), 255, -1)

    if keypoints1 is None:
        keypoints1, descriptors1 = method.detectAndCompute(gray1, mask1)
        mask2 = mask1

    keypoints2, descriptors2 = method.detectAndCompute(gray2, mask2)

    print("\tHledané body vytvořeny.\n")

    if False:
        plt.figure(figsize=(int(0.0017 * width), int(0.0017 * height)))
        plt.title("Keypoints")
        plt.imshow(cv2.drawKeypoints(gray2, keypoints2, gray2), cmap='gray')
        plt.tight_layout()
        plt.axis('equal')
        plt.show()

    # Cyklus na detekci trojúhelníků
    for triangle in mesh.get_cells_type("triangle"):

        triangle_points = mesh.points[triangle]

        mask1 = np.zeros(gray1.shape[:2], dtype=np.uint8)

        # Vykreslení mnohoúhelníků na maskách
        cv2.fillPoly(mask1, [np.array(triangle_points[:, :2], np.int32)], 255)

        # Nalezení klíčových bodů a popisovačů pro oba obrazy - POUZE těch v MASCE1
        selected_keypoints = [
            keypoints1[idx] for idx, kp in enumerate(keypoints1) if mask1[int(kp.pt[1]), int(kp.pt[0])]]
        selected_descriptors = descriptors1[
            [idx for idx, kp in enumerate(keypoints1) if mask1[int(kp.pt[1]), int(kp.pt[0])]]]

        # Porovnání popisovačů pomocí algoritmu BFMatcher
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(selected_descriptors, descriptors2, k=2)

        # Aplikace prahu na shody mezi popisovači
        good_matches = [m for m, n in matches if m.distance < precision * n.distance]

        # seřazení podle přesnosti
        good_matches.sort(key=lambda x: x.distance)

        """matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)"""

        """matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
        matches = matcher.match(selected_descriptors, descriptors2)
        good_matches = matches"""

        # omezení počtu dat podle limitu: "points_limit"
        end = min(points_limit, len(good_matches))
        good_matches = good_matches[:end]

        x_old.extend([selected_keypoints[m.queryIdx].pt[0] for m in good_matches])
        y_old.extend([selected_keypoints[m.queryIdx].pt[1] for m in good_matches])
        x_new.extend([keypoints2[m.trainIdx].pt[0] for m in good_matches])
        y_new.extend([keypoints2[m.trainIdx].pt[1] for m in good_matches])
        # angle_old.extend([selected_keypoints[m.queryIdx].angle for m in good_matches])
        # angle_new.extend([keypoints2[m.trainIdx].angle for m in good_matches])
        index.append(index[counter] + end)

        if counter == 0:
            print("\t\tElement", counter + 1, "hotov.\t\t[", counter + 1, "/", length, "]")
        elif counter % 100 == 0:
            print("\t\tElement", counter, "hotov.\t\t[", counter, "/", length, "]")
        elif counter + 1 == length:
            print("\t\tElement", counter + 1, "hotov.\t\t[", counter + 1, "/", length, "]")

        if False:
            # Vykreslení bodů daného elemenetu
            # Vytvoření seznamu indexů odpovídajících klíčových bodů
            # matching_indices = [m.queryIdx for m in good_matches]

            # Vytvoření seznamu souřadnic odpovídajících good_matches
            matched_keypoints1 = [selected_keypoints[m.queryIdx].pt for m in good_matches]
            matched_keypoints2 = [keypoints2[m.trainIdx].pt for m in good_matches]

            # Převod obrázku z BGR do RGB pro použití s Matplotlib
            image_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

            # Vykreslení bodů na obrázku
            for point in matched_keypoints1:
                plt.plot(point[0], point[1], 'ro', markersize=5)

            # Zobrazení obrázku s vykreslenými body
            plt.imshow(image_rgb)
            plt.axis('off')
            plt.tight_layout()
            plt.show()

            # Vykreslení shod na obrazu
            matched_image = cv2.drawMatches(image1, selected_keypoints, image2, keypoints2, good_matches, None,
                                            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

            # Zobrazení výsledku
            cv2.namedWindow('Matched image', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('Matched image', int(0.25 * width), int(0.25 * height))
            cv2.imshow("Matched image", matched_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        counter += 1

    print("\n\tHledání elementů dokončeno.")

    outcome = np.zeros((len(x_old), 4))
    outcome[:, 0], outcome[:, 1], outcome[:, 2], outcome[:, 3] = x_old, y_old, x_new, y_new
    # outcome[:, 4], outcome[:, 5] = angle_old, angle_new

    return outcome, index

In [ ]:
def mark_points_on_canvas(name):
    marked_points = []
    h_, w_ = int(0.0017 * height), int(0.0017 * width)

    # Vytvoření figure a osy v matplotlib
    figure, axes = plt.subplots(figsize=(w_, h_), num=str(names[name]))
    plt.title("Mark points on image")

    # Funkce pro označení bodů
    def mark_points(event):
        if keyboard.is_pressed('ctrl') and keyboard.is_pressed('space'):
            x = event.xdata
            y = event.ydata
            axes.plot(x, y, 'ro')
            figure.canvas.draw()
            if name == 2 and len(marked_points) > 3:
                plt.close()
            else:
                marked_points.append((x, y))

        if keyboard.is_pressed('escape') or keyboard.is_pressed('enter') and keyboard.is_pressed('space'):
            plt.close()

        if keyboard.is_pressed('c') and keyboard.is_pressed('shift'):
            sys.exit()

    # Připojení události ke funkci mark_points
    cid = figure.canvas.mpl_connect('key_press_event', mark_points)

    # Zobrazení obrázku v matplotlib
    if name == 1:
        # Definice vrcholů prvního mnohoúhelníku
        vertices1 = np.array(points1, np.int32)

        # Vytvoření prázdných mask pro obě oblasti
        mask = np.zeros(image1.shape[:2], dtype=np.uint8)

        # Vykreslení mnohoúhelníků na maskách
        cv2.fillPoly(mask, [vertices1], 255)

        # Aplikace mask na obraz
        masked_image1 = cv2.bitwise_and(image1, image1, mask=mask)

        # Vytvoření obrazu s intenzitou 0.2
        image_half_intensity = (image1 * 0.2).astype(np.uint8)

        # Spojení původního obrazu s intenzitou 0.5 a sledovanými oblastmi
        combined_image = cv2.addWeighted(image_half_intensity, 1.0, masked_image1, 0.5, 0)
        axes.imshow(cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB))

    # Zobrazení první
    else:
        axes.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))

    # Zobrazení obrázku s označenými body
    plt.tight_layout()
    plt.show()

    if name == 2:
        sorted_points = reorder(marked_points).astype(np.int32)
        rectangle_points = np.zeros((4, 2))
        rectangle_points[0, 0] = rectangle_points[3, 0] = (sorted_points[0, 0] + sorted_points[3, 0]) / 2
        rectangle_points[0, 1] = rectangle_points[1, 1] = (sorted_points[0, 1] + sorted_points[1, 1]) / 2
        rectangle_points[1, 0] = rectangle_points[2, 0] = (sorted_points[1, 0] + sorted_points[2, 0]) / 2
        rectangle_points[2, 1] = rectangle_points[3, 1] = (sorted_points[2, 1] + sorted_points[3, 1]) / 2
        marked_points = rectangle_points

    return marked_points

In [ ]:
def pixel_correlation(points):
    x1, y1 = np.min(points, axis=0)
    x2, y2 = np.max(points, axis=0)

    template = gray1[y1:y2, x1:x2]

    # Porovnejte šablonu s druhou fotografií pomocí metody šablony
    result = cv2.matchTemplate(gray2, template, cv2.TM_CCOEFF_NORMED)

    # Práh pro výběr dobrých shod
    threshold = 1

    # Cyklus pro nalezení oblasti snižováním hranice přesnosti
    while True:
        # Naleznete polohy, kde je shoda vyšší než prah
        locations = np.where(result >= threshold)

        found_coordinates = np.array(locations).T
        found_coordinates[:, [0, 1]] = found_coordinates[:, [1, 0]]
        if len(found_coordinates) != 0:
            break
        else:
            threshold = threshold - 0.01
            if threshold < 0.1:
                print("\n\tOblast nenalezena.")
                sys.exit()

    x_f_mean = np.mean(found_coordinates[:, 0])
    y_f_mean = np.mean(found_coordinates[:, 1])

    print("\n\tNalezená oblast.\n\t\tPřesnost:", threshold * 100, "%")

    if False:
        cv2.namedWindow('Original area', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Original area', x2 - x1, y2 - y1)
        cv2.imshow('Original area', template)

        cv2.line(image2, (x1, y1), (x1, int(y_f_mean + y2 - y1)), (255, 0, 0), 5)
        cv2.line(image2, (x2, y1), (x2, int(y_f_mean + y2 - y1)), (255, 0, 0), 5)
        cv2.rectangle(image2, (x1, y1), (x2, y2), (255, 255, 255), 7)
        cv2.rectangle(image2, (int(x_f_mean), int(y_f_mean)), (int(x_f_mean + x2 - x1), int(y_f_mean + y2 - y1)),
                      (0, 255, 0), 5)
        cv2.circle(image2, (int(x_f_mean), int(y_f_mean)), 5, (0, 0, 255), 15)

        cv2.namedWindow('Found area', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Found area', x2 - x1, y2 - y1)
        cv2.imshow('Found area', gray2[int(y_f_mean):int(y_f_mean + y2 - y1), int(x_f_mean):int(x_f_mean + x2 - x1)])

        cv2.namedWindow('Found image', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('Found image', int(0.25 * width), int(0.25 * height))

        print("\n\t\tPress Enter or Space to close the windows.")

        # Zobrazení výsledku
        cv2.imshow('Found image', image2)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    upper_area = np.array([[x_f_mean, y_f_mean], [int(x_f_mean + x2 - x1), int(y_f_mean + y2 - y1)]])

    return upper_area

In [ ]:
def results(result, old_center, limit, mesh, upper_area_cor):
    print("\nVyhodnocování výsledků.")

    new_center = np.empty((0, 2))

    tri_index = mesh.get_cells_type("triangle")

    triangle_points_old = mesh.points[:, :2]

    tri_cor_old = triangle_points_old[tri_index]
    tri_cor_new = np.empty((0, 3, 2))

    delete_indexes = []

    for i in range(len(limit) - 1):
        if limit[i] == limit[i + 1]:
            delete_indexes.append(i)
            continue

        segment = result[limit[i]:limit[i + 1], :4]
        size_segment = len(segment)

        """
        def calculate_angle(point0, points):
            vector = points - point0  # Rozdíl mezi prvním bodem a ostatními body
            angles = np.arctan2(vector[:, 1], vector[:, 0])  # Výpočet úhlu vzhledem k ose x
            return angles

        def rotation_matrix(angle):
            return np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])

        # Příklad použití
        original_points = segment[:, 0:2]
        original_center = old_center[i]
        new_points = segment[:, 2:4]

        original_angles = calculate_angle(original_points[0], original_points[1:])
        new_angles = calculate_angle(new_points[0], new_points[1:])

        new_position = [np.mean(new_points - original_points, axis=0)]
        a = [new_points[0] - original_points[0]]

        rotation_angle = np.mean(new_angles - original_angles)

        rot_pos = np.dot(original_center, rotation_matrix(rotation_angle))

        new_centers = new_position + rot_pos
        new_center = np.append(new_center, new_centers, axis=0)"""

        """if size_segment > 5:
            index_1, index_2 = int(size_segment * 0.2), int(size_segment * 0.4)
            index_3, index_4 = int(size_segment * 0.6), int(size_segment * 0.8)

        elif size_segment > 2:
            index_1 = 0
            index_2 = int(size_segment * 0.5)
            index_3 = int(size_segment * 0.5)
            index_4 = -1
        else:
            index_1 = index_2 = 0
            index_3 = index_4 = -1

        vector_members_old = segment[[0, index_1, index_2, index_3, index_4, -1]][:, [0, 1]]
        vector_members_new = segment[[0, index_1, index_2, index_3, index_4, -1]][:, [2, 3]]
        distances = np.linalg.norm(vector_members_old - old_center[i], axis=1)"""

        # Definice kružnic
        inputs = np.zeros((size_segment, 3))
        inputs[:, 0:2], inputs[:, 2], = segment[:, 2:4], np.linalg.norm(segment[:, :2] - old_center[i], axis=1)

        # Vypočet průsečíku a uložení
        current_center = transformation(segment[:, 0:2], segment[:, 2:4], old_center[i]).reshape(1, 2)
        new_center = np.append(new_center, current_center, axis=0)
        """current_center = circle_intersection(inputs).reshape(1, 2)
        current_mean = np.mean(segment[:, 2:4], axis=0)
        calculation_type = np.linalg.norm(current_center - current_mean) < np.mean(inputs[:, 2])
        if calculation_type:
            new_center = np.append(new_center, current_center, axis=0)
        else:
            new_center = np.append(new_center, second_circle_intersection(inputs, current_mean).reshape(1, 2), axis=0)"""

        data = np.zeros((3, 2))
        for k in range(3):
            # Definice kružnic
            inputs = np.zeros((size_segment, 3))
            inputs[:, 0:2] = segment[:, 2:4]
            inputs[:, 2] = np.linalg.norm(segment[:, :2] - tri_cor_old[i, k, :], axis=1)

            # Vypočet průsečíku a uložení
            data[k] = transformation(segment[:, 0:2], segment[:, 2:4], tri_cor_old[i, k, :]).reshape(1, 2)
            """if calculation_type:
                data[k] = circle_intersection(inputs).reshape(1, 2)
            else:
                data[k] = second_circle_intersection(inputs, current_mean).reshape(1, 2)"""
        tri_cor_new = np.append(tri_cor_new, [data], axis=0)

        if False:  # i == 40:
            original_points = segment[:, 0:2]
            original_center = old_center[i]
            new_points = segment[:, 2:4]

            plt.figure()
            fig.title("Centers of " + str(i) + ". element")
            plt.scatter(original_points[:, 0], original_points[:, 1], c='blue', marker='o', label='Původní body')
            plt.scatter(new_points[:, 0], new_points[:, 1], c='red', marker='o', label='Původní body')
            plt.scatter(original_center[0], original_center[1], c='green', marker='s', label='Původní těžiště')
            plt.scatter(new_center[i, 0], new_center[i, 1], c='black', marker='s', label='Nové těžiště')
            plt.axis('equal')
            plt.legend()
            plt.tight_layout()
            plt.show()

        # print(i, ":", limit[i + 1] - limit[i])

    tri_index = np.delete(tri_index, delete_indexes, axis=0)
    del delete_indexes

    """upper_area_elements = np.sum(old_center[:, 1] < 900)
    upper_area_lines = np.sum(result[:, 1] < 900)
    print("\n\tPočet horních elementů:", upper_area_elements, " , Počet řádků pro horní elementy", upper_area_lines)"""

    sx_old, sy_old, sx_new, sy_new = np.array([]), np.array([]), np.array([]), np.array([])
    # rot_x, rot_y = np.array([]), np.array([])

    for i in range(len(limit) - 1):
        if limit[i] == limit[i + 1]:
            continue

        segment = result[limit[i]:limit[i + 1]]
        sx_old = np.append(sx_old, np.mean(segment[:, 0], axis=0))
        sy_old = np.append(sy_old, np.mean(segment[:, 1], axis=0))
        sx_new = np.append(sx_new, np.mean(segment[:, 2], axis=0))
        sy_new = np.append(sy_new, np.mean(segment[:, 3], axis=0))

        """new_x = np.mean(segment[:, 2], axis=0) + (center[i, 0] - np.mean(segment[:, 0], axis=0))
        new_y = np.mean(segment[:, 3], axis=0) + (center[i, 1] - np.mean(segment[:, 1], axis=0))
        angle_rad = np.radians(result[i, 5]-result[i, 4])
        rot_x = np.append(rot_x, new_x * np.cos(angle_rad) - new_y * np.sin(angle_rad))
        rot_y = np.append(rot_y, new_x * np.sin(angle_rad) + new_y * np.cos(angle_rad))

    # s = np.stack((sx_old, sy_old, sx_new, sy_new), axis=1)"""

    print("\n\tÚspěšně nalezeno  {", len(new_center), "/", len(old_center), "}  elementů.")

    """# Vytvoříme prázdné pole pro nové vrcholy
    next_data1 = np.zeros_like(triangle_points_old)
    next_data1[tri_index] = tri_cor_new"""

    # Vytvoření nových trojúhelníků průměrem hodnot jednotlyvých elementů
    next_data = np.zeros_like(triangle_points_old, dtype=np.float64)
    np.add.at(next_data, np.int32(tri_index), tri_cor_new)
    quantity = np.bincount(np.int32(tri_index).ravel())

    # np.seterr(divide='ignore', invalid='ignore')  # ignorování chybových hlášek
    nonzero_indices = (quantity != 0)  # Kontrola na neplatné hodnoty před dělením
    try:
        next_data[nonzero_indices] /= quantity[nonzero_indices, np.newaxis]  # průměrování souřadnic počtem výskytu
    except IndexError:
        next_data = next_data[:len(quantity)]
        next_data[nonzero_indices] /= quantity[nonzero_indices, np.newaxis]
        print("\n\tChyba, seznam byl zkrácen.\n")
    # np.seterr(divide='warn', invalid='warn')  # obnovení chybových hlášek

    """wrong_list, wrong_index = [], []
    up_limit, down_limit, right_limit, left_limit = int(min(points1[:, 1]) + 600), int(max(points1[:, 1]) + 50), \
        int(max(points1[:, 0]) + 600), int(min(points1[:, 0]) - 600)

    for t in range(len(new_center)):
        up_points = new_center[t, 1] < up_limit
        down_points = new_center[t, 1] > down_limit
        right_points = new_center[t, 0] > right_limit
        left_points = new_center[t, 0] < left_limit
        if up_points or down_points or right_points or left_points:
            wrong_index.append(t)
            wrong_list.append(limit[t + 1] - limit[t])
    if len(wrong_list) > 0:
        print("\n\tIndexy špatných bodů:", wrong_index)
        print("\tPočet špatných bodů:", wrong_list)
    else:
        del wrong_list, wrong_index
    del up_limit, down_limit, right_limit, left_limit"""

    difference_wrong = np.linalg.norm(new_center - np.array([sx_new, sy_new]).T, axis=1)
    # difference_wrong = np.linalg.norm(np.mean(new_center) - inputs[:, :2], axis=1)
    indices_wrong = np.where(np.mean(difference_wrong) >= np.mean(inputs[:, 2]), 1, 0)
    indices_wrong = np.nonzero(indices_wrong)[0]
    distances_wrong = difference_wrong[indices_wrong]
    print("\n\tPočet špatných bodů:", len(indices_wrong),
          "\n\t\t-     Indexy:", indices_wrong,
          "\n\t\t- Vzdálenost:", distances_wrong)
    # print(new_center[indices_wrong])
    # print(np.array([sx_new, sy_new]).T[indices_wrong])

    if False:
        fig, ax = plt.subplots(figsize=(int(0.0017 * width), int(0.0017 * height)))
        plt.title("Triangle elements")
        plt.imshow(gray2, cmap='gray')

        plt.triplot(next_data[:, 0], next_data[:, 1], tri_index, color='green')

        plt.scatter(sx_new, sy_new,
                    s=5,  # Velikost bodů
                    c='green',  # Barva bodů
                    marker='s'  # Znak bodů
                    )

        plt.scatter(new_center[:, 0], new_center[:, 1],
                    s=5,  # Velikost bodů
                    c='orange',  # Barva bodů
                    marker='o'  # Znak bodů
                    )
        # Vykreslení obdélníka
        rec_w, rec_h = upper_area_cor[1, 0] - upper_area_cor[0, 0], upper_area_cor[1, 1] - upper_area_cor[0, 1]
        rect_patch = Rectangle((upper_area_cor[0]), rec_w, rec_h, edgecolor='firebrick', linewidth=1.5,
                               facecolor='none')
        ax.add_patch(rect_patch)
        rect_patch = Rectangle((upper_area_cor[0]), rec_w, rec_h, facecolor='red', alpha=0.1)
        ax.add_patch(rect_patch)

        plt.axis('equal')
        plt.tight_layout()
        plt.show()

    if False:
        # fig, ax = plt.subplots(figsize=(int(0.0017 * width), int(0.0017 * height)))
        if proces < 25:
            ax = plt.subplot(min(5, graph_count), min(5, graph_count), min(proces + 1, 25))
        if proces == 25:
            plt.pause(0.5)
            plt.figure(figsize=(15, 7))
            plt.pause(2)
        if proces > 24:
            ax = plt.subplot(min(5, mgraph_count), min(5, graph_count), proces - 24)

        plt.title('Triangle elements - Image {}.'.format(proces + 1))
        # plt.title("Triangle elements")
        plt.imshow(gray2, cmap='gray')
        # plt.triplot(tri_cor_new[:, 0], tri_cor_new[:, 1], mesh.get_cells_type("triangle"))
        """patches = [Polygon(triangle, closed=True, fill=None, color='royalblue') for triangle in tri_cor_new]
        ax.patches.extend(patches)"""

        plt.triplot(next_data[:, 0], next_data[:, 1], tri_index, color='green')
        # plt.triplot(next_data1[:, 0], next_data1[:, 1], tri_index, color='red')

        plt.scatter(sx_new, sy_new,
                    s=5,  # Velikost bodů
                    c='green',  # Barva bodů
                    marker='s'  # Znak bodů
                    )

        plt.scatter(new_center[:, 0], new_center[:, 1],
                    s=5,  # Velikost bodů
                    c='orange',  # Barva bodů
                    marker='o'  # Znak bodů
                    )
        # Vykreslení obdélníka
        rec_w, rec_h = upper_area_cor[1, 0] - upper_area_cor[0, 0], upper_area_cor[1, 1] - upper_area_cor[0, 1]
        rect_patch = Rectangle((upper_area_cor[0]), rec_w, rec_h, edgecolor='firebrick', linewidth=1.5,
                               facecolor='none')
        ax.add_patch(rect_patch)
        rect_patch = Rectangle((upper_area_cor[0]), rec_w, rec_h, facecolor='red', alpha=0.1)
        ax.add_patch(rect_patch)

        plt.axis('equal')
        plt.tight_layout()

        # Pauza pro zobrazení grafu
        plt.pause(0.5)
        if proces == len(image_files) - 1:
            plt.show(block=True)
        else:
            plt.show(block=False)
        plt.pause(2)

        plt.close()

    if False:
        # Vykreslení těžišť - původní a nové nalezené
        plt.figure(figsize=(int(0.0017 * width), int(0.0017 * height)))
        plt.title("New and old element centers")
        plt.subplot(1, 2, 1)
        plt.imshow(gray1, cmap='gray')
        plt.scatter(old_center[:, 0], old_center[:, 1],
                    s=20,  # Velikost bodů
                    c='orange',  # Barva bodů
                    marker='o'  # Znak bodů
                    )

        plt.tight_layout()

        plt.subplot(1, 2, 2)
        plt.imshow(gray2, cmap='gray')
        plt.scatter(new_center[:, 0], new_center[:, 1],
                    s=20,  # Velikost bodů
                    c='orange',  # Barva bodů
                    marker='o'  # Znak bodů
                    )

        plt.tight_layout()

        plt.show()

        print("Výsledky vhodnoceny.")

    return next_data, new_center, tri_index, tri_cor_new, indices_wrong

In [ ]:
def set_ROI():
    global points1, points2, points3, names

    names = ['Přidání bodů positivní oblasti', 'Přidání bodů negativní oblast', 'Fig']

    if mark_points_by_hand:
        while True:
            points = mark_points_on_canvas(0)
            points1 = np.array(points, np.int32)
            if len(points1) > 2:
                break

        points2 = []
        points = mark_points_on_canvas(1)
        points2 = np.array(points, np.int32)

        while True:
            points = mark_points_on_canvas(2)
            points3 = np.array(points, np.int32)
            if len(points1) > 3:
                break
    else:
        points1 = np.array([[2492, 1113], [1902, 2125], [2492, 3151],
                            [3666, 3151], [4257, 2137], [3666, 1113]], np.int32)

        points2 = np.array([[2523, 1165], [2272, 1595], [3203, 2142], [3173, 2185], [2248, 1648], [1965, 2128],
                            [2523, 3097], [3635, 3097], [3883, 2666], [2954, 2125], [2982, 2081], [3913, 2619],
                            [4192, 2138], [3635, 1165]], np.int32)

        points3 = np.array([[2600, 400], [3450, 400], [3450, 650], [2600, 650]], np.int32)

    found_mesh, found_triangle_centers = divide_image(points1, points2, size)

    del names
    return found_mesh, found_triangle_centers

In [ ]:
def reorder(points):
    points = np.array(points)
    # Vypočítání středu bodů
    center = np.mean(points, axis=0)

    # Výpočet úhlů bodů od středu
    angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])

    # Seřazení bodů podle úhlů
    sorted_indices = np.argsort(angles)

    # Přeuspořádání bodů
    sorted_points = points[sorted_indices]

    # Přesunutí prvního bodu na začátek
    first_index = np.where(sorted_indices == 0)[0][0]
    sorted_points = np.roll(sorted_points, -first_index, axis=0)
    sorted_points = np.array(sorted_points, dtype=np.float32).reshape((4, 2))

    return sorted_points

In [ ]:
def divide_image(area1, area2=None, mesh_size=300):
    with pygmsh.occ.Geometry() as geom:
        geom.characteristic_length_max = mesh_size
        geom.characteristic_length_min = mesh_size

        """point_entities = [geom.add_point(point, mesh_size) for point in area1]
        point_entities.append(point_entities[0])
        s1 = geom.add_spline(point_entities)
        l1 = geom.add_curve_loop([s1])
        poly1 = geom.add_plane_surface(l1)"""

        poly1 = geom.add_polygon(
            area1,  # mesh_size=mesh_size,
        )

        if area2 is not None and len(area2) > 2:
            """point_entities = [geom.add_point(point, mesh_size) for point in area2]
            point_entities.append(point_entities[0])
            s2 = geom.add_spline(point_entities)
            l2 = geom.add_curve_loop([s2])
            poly2 = geom.add_plane_surface(l2)"""

            poly2 = geom.add_polygon(
                area2,  # mesh_size=0.1,
            )

            geom.boolean_difference(poly1, poly2)

        mesh = geom.generate_mesh(dim=2)

        print("\n\tTriangulation is done.\n")

    triangle_centers = np.mean(mesh.points[mesh.get_cells_type("triangle")][:, :, :2], axis=1)

    print("\tVytvořeno", len(triangle_centers), "elementů.")

    if True:
        # Obraz elemntů na fotografii
        # Vytvoření figure a osy v matplotlib
        plt.figure(figsize=(int(0.0017 * width), int(0.001 * height)))
        plt.suptitle("Triangled ROI area")
        # Vytvoření GridSpecu
        gs = GridSpec(2, 7)

        # Definice umístění pro každý graf
        ax1 = plt.subplot(gs[:, :5])
        ax2 = plt.subplot(gs[0, 5:])
        ax3 = plt.subplot(gs[1, 5:])

        ax1.imshow(gray1, cmap='gray')
        ax1.triplot(mesh.points[:, 0], mesh.points[:, 1], mesh.get_cells_type("triangle"))
        #                                                mesh.get_cells_type("triangle") = mesh.cells[1].data
        ax1.scatter(triangle_centers[:, 0], triangle_centers[:, 1],
                    s=5,  # Velikost bodů
                    c='orange',  # Barva bodů
                    marker='o'  # Znak bodů
                    )
        ax1.axis('equal')
        ax1.set_xlim(0, width)
        ax1.set_ylim(height, 0)

        print("\t\tFirst graph: finished.")

        # Obraz barevných elementů
        # Vykreslení všech trojúhelníků
        [ax2.triplot(mesh.points[cell][:, 0], mesh.points[cell][:, 1]) for cell in mesh.get_cells_type("triangle")]
        ax2.scatter(triangle_centers[:, 0], triangle_centers[:, 1],
                    s=20,  # Velikost bodů
                    c='orange',  # Barva bodů
                    marker='o'  # Znak bodů
                    )
        plt.gca().invert_yaxis()
        ax2.axis('equal')

        print("\t\tSecond graph: finished.")

        # Obraz jednoho maskovaného elementu
        triangle = min(10, len(mesh.get_cells_type("triangle")))  # který trojúhelník chci vykreslit

        triangle_points = mesh.points[mesh.get_cells_type("triangle")[triangle]]

        # Vytvoření prázdných mask pro obě oblasti
        mask = np.zeros(gray1.shape[:2], dtype=np.uint8)

        # Vykreslení mnohoúhelníků na maskách
        cv2.fillPoly(mask, [np.array(triangle_points[:, :2], np.int32)], 255)

        x_max, x_min = int(max(triangle_points[:, 0]) + 10), int(min(triangle_points[:, 0]) - 10)
        y_max, y_min = int(max(triangle_points[:, 1]) + 10), int(min(triangle_points[:, 1]) - 10)
        masked_image = cv2.bitwise_and(gray1, gray1, mask=mask)[y_min:y_max, x_min:x_max]

        ax3.imshow(masked_image, cmap='gray')
        ax3.scatter(triangle_centers[triangle, 0] - x_min, triangle_centers[triangle, 1] - y_min,
                    s=50,  # Velikost bodů
                    c='red',  # Barva bodů
                    marker='s'  # Znak bodů
                    )
        ax3.axis('off')
        ax3.axis('equal')

        print("\t\tThird graph: finished.\n")
        plt.tight_layout()
        plt.show()

    return mesh, triangle_centers

In [ ]:
####################
# ### SETTINGS ### #

mark_points_by_hand = False

main_image_folder = 'photos'

size = 140  # !=_ 135 _=!,   100, 85 - min,   (40)
make_gray = True

points_limit = 15
precision = 0.65

show_final_image = -1  # Kterou fotografii vykreslit

file_list = os.listdir(main_image_folder)
image_files = [f for f in file_list if os.path.isfile(os.path.join(main_image_folder, f)) and
               f.lower().endswith((".jpg", ".jpeg", ".png", ".JPG"))]

# Omezeni počtu snímků
image_files = image_files[:]  # načátání snímků (první je 0) př: "image_files[2:5] od 2 do 5"
image_files = [image_files[0], image_files[-1]]

print("\nSpuštění programu pro detekci fotek.",
      "\n\n\tNastavení:",
      "\n\t\tAktuální složka:", '"', main_image_folder, '"',
      "\n\t\tVelikost elementů:", size,
      "\n\t\tMinimální počet bodů:", points_limit,
      "\n\t\tPřesnost nalezených bodů:", precision, "\n")

In [ ]:
triangle_vertices_all, triangle_centers_all, triangle_indexes_all, triangle_points_all, \
    correlation_area_points_all, wrong_points_indexes_all = [], [], [], [], [], []

# Načtení první a druhé fotografie
image1 = cv2.imread(os.path.join(main_image_folder, image_files[0]))
keypoints1 = descriptors1 = None

# Vytvoření okna pro všechny grafy
"""plt.figure(figsize=(15, 7))"""
graph_count = int(np.ceil(np.sqrt(len(image_files))))

# Získání počtu kanálů obrázku
# Pokud je počet kanálů roven 1, pak je obrázek ve stupních šedi
# Převod obou fotografií na šedotónový formát
if make_gray:
    if image1.shape[-1] != 1:
        gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
else:
    gray1 = image1.copy()

# Naleznete rozměry první fotografie
height, width = gray1.shape

m, c = set_ROI()

for proces in range(0, len(image_files)):

    print("\n=================================================================",
          "\n=================================================================",
          "\nAktuální proces:  [", proces + 1, "/", len(image_files), "]\t  Fotografie:", image_files[proces])

    image2 = cv2.imread(os.path.join(main_image_folder, image_files[proces]))

    if make_gray:
        if image2.shape[-1] != 1:
            gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    else:
        gray2 = image2.copy()

    correlation_area_points = pixel_correlation(points3)
    shift = correlation_area_points[0, 1] - points3[0, 1]
    r, i = point_track(mesh=m, current_shift=shift)

    triangle_vertices, triangle_centers, triangle_indexes, triangle_points, wrong_points_indexes \
        = results(r, c, i, m, correlation_area_points)

    triangle_vertices_all.append(triangle_vertices)  # n,2
    triangle_centers_all.append(triangle_centers)  # n,2
    triangle_indexes_all.append(triangle_indexes)  # n,3
    triangle_points_all.append(triangle_points)  # n,3,2
    wrong_points_indexes_all.append(wrong_points_indexes)  # n
    correlation_area_points_all.append(correlation_area_points)  # n,2

show_results_graph(show_final_image)  # Vykreslení výsledného grafu fotografie

In [ ]:
triangle_points1 = triangle_points_all[0]
mask1 = np.zeros(gray1.shape[:2], dtype=np.uint8)
mask2 = mask1.copy()

j = 1
i = 1

triangle_coordinates1 = triangle_points1[i]
cv2.fillPoly(mask1, [np.int32(triangle_coordinates1)], 255)
masked_image1 = cv2.bitwise_and(gray1, gray1, mask=mask1)

triangle_points2 = triangle_points_all[j]
triangle_coordinates2 = triangle_points2[i]
cv2.fillPoly(mask2, [np.int32(triangle_coordinates2)], 255)
masked_image2 = cv2.bitwise_and(gray2, gray2, mask=mask2)

fine_mesh, fine_mesh_centers = divide_image(triangle_coordinates1, mesh_size=10)
# keypoints1 = None
# point_track(mesh=fine_mesh, current_shift=None, m1=mask1, m2=mask2)

fine_triangle_points = fine_mesh.points[fine_mesh.get_cells_type("triangle")][0]
fine_triangle_points = fine_triangle_points[:, :2].reshape(3, 2)
mask = np.zeros(gray1.shape[:2], dtype=np.uint8)
cv2.fillPoly(mask, [np.int32(points3)], 255)
t = cv2.bitwise_and(gray1, gray1, mask=mask)[350:700, 2550:3500]

plt.figure()
plt.imshow(cv2.cvtColor(t, cv2.COLOR_BGR2RGB))
plt.show()

result = cv2.matchTemplate(gray1, t, cv2.TM_SQDIFF_NORMED)
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
top_left = max_loc
bottom_right = (top_left[0] + t.shape[1], top_left[1] + t.shape[0])

print(top_left, bottom_right)

plt.figure()
cv2.rectangle(image1, top_left, bottom_right, (0, 0, 255), 10)
plt.imshow(cv2.cvtColor(image1, cv2.COLOR_BGR2RGB))
plt.show()

# Načtěte původní a cílovou fotografii
img1 = gray1
img2 = gray2

# Detekce klíčových bodů a výpočet deskriptorů
orb = cv2.ORB_create()  # ####################### #TODO #################################### Změnit na Shift
keypoints1, descriptors1 = orb.detectAndCompute(img1, mask1)
keypoints2, descriptors2 = orb.detectAndCompute(img2, mask2)

if False:
    plt.figure(figsize=(int(0.0017 * width), int(0.001 * height)))
    plt.subplot(121)
    plt.title("Keypoints 1")
    plt.imshow(cv2.drawKeypoints(img1, keypoints1, img1), cmap='gray')
    plt.axis('equal')
    plt.tight_layout()

    plt.subplot(122)
    plt.title("Keypoints 2")
    plt.imshow(cv2.drawKeypoints(img2, keypoints2, img2), cmap='gray')
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

# Shoda klíčových bodů
bf = cv2.BFMatcher()
matches = bf.knnMatch(descriptors1, descriptors2, k=2)  # ORB 141, 56
good_matches = [m for m, n in matches if m.distance < precision * n.distance]  # ORB 36, 40

# Vybrání shodných klíčových bodů
src_pts = np.float32([keypoints1[match.queryIdx].pt for match in good_matches]).reshape(-1, 2)
dst_pts = np.float32([keypoints2[match.trainIdx].pt for match in good_matches]).reshape(-1, 2)

# Výpočet transformační matice
M, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)

x_max, x_min = int(max(src_pts[:, 0]) + 50), int(min(src_pts[:, 0]) - 50)
y_max, y_min = int(max(src_pts[:, 1]) + 50), int(min(src_pts[:, 1]) - 50)

# Zobrazení výsledku
plt.figure()
plt.subplot(121)
plt.imshow(img1[y_min:y_max, x_min:x_max], cmap='gray')
plt.axis('equal')
plt.tight_layout()

# Aplikace transformace na trojúhelníkovou oblast
triangle_pts = np.float32([dst_pts]).reshape(-1, 1, 2)
transformed_pts = cv2.perspectiveTransform(triangle_pts, M).reshape(triangle_pts.shape[0], -1)

x_max, x_min = int(max(transformed_pts[:, 0]) + 50), int(min(transformed_pts[:, 0]) - 50)
y_max, y_min = int(max(transformed_pts[:, 1]) + 50), int(min(transformed_pts[:, 1]) - 50)

transformed_img = cv2.warpPerspective(img2, M, (img1.shape[1], img1.shape[0]))

# Vykreslení trojúhelníku na cílové fotografii
cv2.polylines(transformed_img, [np.int32(transformed_pts)], True, (0, 255, 0), 2)

plt.subplot(122)
plt.imshow(transformed_img[y_min:y_max, x_min:x_max], cmap='gray')
plt.axis('equal')
plt.tight_layout()
plt.show()

print("\nJemné prohledávání není zcela implementováno.")

In [ ]:
triangle_points1 = triangle_points_all[0]
mask1 = np.zeros(gray1.shape[:2], dtype=np.uint8)
mask2 = mask1.copy()

j = 1
i = 1

triangle_coordinates1 = triangle_points1[i]
cv2.fillPoly(mask1, [np.int32(triangle_coordinates1)], 255)
masked_image1 = cv2.bitwise_and(gray1, gray1, mask=mask1)

triangle_points2 = triangle_points_all[j]
triangle_coordinates2 = triangle_points2[i]
cv2.fillPoly(mask2, [np.int32(triangle_coordinates2)], 255)
masked_image2 = cv2.bitwise_and(gray2, gray2, mask=mask2)

fine_mesh, fine_mesh_centers = divide_image(triangle_coordinates1, mesh_size=10)

fine_triangle_points = fine_mesh.points[fine_mesh.get_cells_type("triangle")][-3]
fine_triangle_points = np.int32(fine_triangle_points[:, :2]).reshape(3, 2)
x, y, w, h = cv2.boundingRect(fine_triangle_points)

region_of_interest = gray1[y:y + h, x:x + w]

plt.figure()
plt.imshow(region_of_interest, cmap='gray')
plt.show()

a = np.int32(min(triangle_coordinates1[:, 0] * 0.9))
b = np.int32(max(triangle_coordinates1[:, 0] * 1.1))
c = np.int32(min(triangle_coordinates1[:, 1] * 0.9))
d = np.int32(max(triangle_coordinates1[:, 1] * 1.1))

result = cv2.matchTemplate(gray1[c:d, a:b], region_of_interest, cv2.TM_CCOEFF_NORMED)
# TM_CCOEFF_NORMED / TM_CCORR_NORMED

min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
top_left = max_loc[0] + a, max_loc[1] + c
bottom_right = (top_left[0] + w, top_left[1] + h)

print((x, y), (x + w, y + h))
print(top_left, bottom_right)

plt.figure()
plt.imshow(gray1[top_left[1]:top_left[1] + h, top_left[0]:top_left[0] + w], cmap='gray')
plt.show()

img = image1.copy()
plt.figure()
cv2.rectangle(img, top_left, bottom_right, (0, 0, 255), 10)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.tight_layout()
plt.show()

In [ ]:
def detect_and_draw_keypoints(image):
    # Vytvoření ORB detektoru
    method = cv2.SIFT_create()

    # Detekce klíčových bodů a popisů
    keypoints, descriptors = method.detectAndCompute(image, None)

    # Vykreslení klíčových bodů na obrázku
    img_with_keypoints = cv2.drawKeypoints(image, keypoints, None)

    return img_with_keypoints, keypoints


def count_keypoints_in_region(keypoints, x_min, y_min, x_max, y_max):
    count = 0
    for kp in keypoints:
        x, y = kp.pt
        if x_min < x < x_max and y_min < y < y_max:
            count += 1
    return count


# Načtení fotografie
image = gray1

# Vytvoření bodů a získání klíčových bodů
img_with_keypoints, keypoints = detect_and_draw_keypoints(image)

# Určení oblasti pro počítání bodů (zde 0:0 až 100:100)
x_min, y_min, x_max, y_max = 2700, 450, 2750, 500

# Sečtení počtu bodů v dané oblasti
keypoints_count_in_region = count_keypoints_in_region(keypoints, x_min, y_min, x_max, y_max)

# Vykreslení obrazu s klíčovými body
plt.figure()
plt.imshow(img_with_keypoints[y_min:y_max, x_min:x_max])
plt.show()

# Výpis počtu bodů v dané oblasti
print("Počet bodů v dané oblasti:", keypoints_count_in_region)

# Načtení fotografie
image = gray1[y_min:y_max, x_min:x_max]

# Vytvoření bodů a získání klíčových bodů
img_with_keypoints, keypoints = detect_and_draw_keypoints(image)

# Určení oblasti pro počítání bodů (zde 0:0 až 100:100)
x_min, y_min, x_max, y_max = 0, 0, image.shape[0], image.shape[1]

# Sečtení počtu bodů v dané oblasti
keypoints_count_in_region = count_keypoints_in_region(keypoints, x_min, y_min, x_max, y_max)

# Vykreslení obrazu s klíčovými body
plt.figure()
plt.imshow(img_with_keypoints)
plt.show()

# Výpis počtu bodů v dané oblasti
print("Počet bodů v dané oblasti:", keypoints_count_in_region)


In [ ]:
def point_track(mesh, current_shift=None, m1=None, m2=None, method="all"):
    global make_first_photo, keypoints1_shift, descriptors1_shift, keypoints1_orb, descriptors1_orb, \
        keypoints2_shift, descriptors2_shift, keypoints2_orb, descriptors2_orb

    shift = cv2.SIFT_create(
        nfeatures=0,  # __________________ Počet detekovaných rysů (0 = všechny dostupné) ______ def = 0
        nOctaveLayers=3,  # ______________ Počet vrstev v každé oktávě _________________________ def = 3
        contrastThreshold=0.04,  # _______ Práh kontrastu pro platnost rysu ____________________ def = 0.04
        edgeThreshold=10,  # _____________ Práh hrany pro platnost rysu blízko k okraji ________ def = 10
        sigma=1.6  # _____________________ Gaussovská hladina oktáv ____________________________ def = 1.6
    )

    orb = cv2.ORB_create()

    x_old, y_old, x_new, y_new, angle_old, angle_new, index = [], [], [], [], [], [], [0]
    counter = 0

    length = len(mesh.get_cells_type("triangle"))

    print("\n\tVytváření hledaných bodů.")

    if isinstance(m1, np.ndarray) and isinstance(m2, np.ndarray):
        mask1 = m1
        mask2 = m2
    else:
        mask1 = np.zeros(gray1.shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask1, [points1], 255)
        try:
            cv2.fillPoly(mask1, [points2], 0)
        except cv2.error:
            pass

        mask2 = np.zeros(gray2.shape[:2], dtype=np.uint8)
        cv2.rectangle(mask2, (1400, int(1050 + current_shift)), (4600, 3200), 255, -1)
        # cv2.rectangle(mask2, (ROI_area[0] - 50, ROI_area[1] + 50), (ROI_area[2] - 50, ROI_area[3] + 50), 255, -1)

    shift_method = True
    orb_method = True

    if method == "all":
        print("Pro výpočet použity metody SHIFT a ORB")
    elif method == "shift":
        orb_method = False
        print("Pro výpočet použita metoda SHIFT")
    elif method == "shift":
        shift_method = False
        print("Pro výpočet použita metoda ORB")
    else:
        print("Špatně zvolena metoda výpočtu, budou použity obě.")

    if make_first_photo:
        mask2 = mask1
        make_first_photo = False
        if shift_method:
            keypoints1_shift, descriptors1_shift = shift.detectAndCompute(gray1, mask1)
        if orb_method:
            keypoints1_orb, descriptors1_orb = orb.detectAndCompute(gray1, mask1)

    if shift_method:
        keypoints2_shift, descriptors2_shift = shift.detectAndCompute(gray2, mask2)
    if orb_method:
        keypoints2_orb, descriptors2_orb = orb.detectAndCompute(gray2, mask2)

    print("\tHledané body vytvořeny.\n")

    if False:
        plt.figure(figsize=(int(0.0017 * width), int(0.0017 * height)))
        plt.title("Keypoints")
        plt.imshow(cv2.drawKeypoints(gray2, keypoints2, gray2), cmap='gray')
        plt.tight_layout()
        plt.axis('equal')
        plt.show()

    # Cyklus na detekci trojúhelníků
    for triangle in mesh.get_cells_type("triangle"):

        triangle_points = mesh.points[triangle]

        mask1 = np.zeros(gray1.shape[:2], dtype=np.uint8)

        # Vykreslení mnohoúhelníků na maskách
        cv2.fillPoly(mask1, [np.array(triangle_points[:, :2], np.int32)], 255)

        # Nalezení klíčových bodů a popisovačů pro oba obrazy - POUZE těch v MASCE1
        selected_keypoints_shift = [
            keypoints1_shift[idx] for idx, kp in enumerate(keypoints1_shift) if mask1[int(kp.pt[1]), int(kp.pt[0])]]
        selected_descriptors_shift = descriptors1_shift[
            [idx for idx, kp in enumerate(keypoints1) if mask1[int(kp.pt[1]), int(kp.pt[0])]]]

        selected_keypoints_orb = [
            keypoints1_orb[idx] for idx, kp in enumerate(keypoints1_orb) if mask1[int(kp.pt[1]), int(kp.pt[0])]]
        selected_descriptors_orb = descriptors1_orb[
            [idx for idx, kp in enumerate(keypoints1) if mask1[int(kp.pt[1]), int(kp.pt[0])]]]

        # Porovnání popisovačů pomocí algoritmu BFMatcher
        bf = cv2.BFMatcher()
        matches_shift = bf.knnMatch(selected_descriptors_shift, descriptors2_shift, k=2)
        matches_orb = bf.knnMatch(selected_descriptors_orb, descriptors2_orb, k=2)
        selected_keypoints = selected_keypoints_shift + selected_keypoints_orb
        matches = matches_shift + matches_orb

        # Aplikace prahu na shody mezi popisovači
        good_matches = [m for m, n in matches if m.distance < precision * n.distance]

        # seřazení podle přesnosti
        good_matches.sort(key=lambda x: x.distance)

        """matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
        matches = matcher.knnMatch(descriptors1, descriptors2, k=2)"""

        """matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
        matches = matcher.match(selected_descriptors, descriptors2)
        good_matches = matches"""

        # omezení počtu dat podle limitu: "points_limit"
        end = min(points_limit, len(good_matches))
        good_matches = good_matches[:end]

        x_old.extend([selected_keypoints[m.queryIdx].pt[0] for m in good_matches])
        y_old.extend([selected_keypoints[m.queryIdx].pt[1] for m in good_matches])
        x_new.extend([keypoints2[m.trainIdx].pt[0] for m in good_matches])
        y_new.extend([keypoints2[m.trainIdx].pt[1] for m in good_matches])
        # angle_old.extend([selected_keypoints[m.queryIdx].angle for m in good_matches])
        # angle_new.extend([keypoints2[m.trainIdx].angle for m in good_matches])
        index.append(index[counter] + end)

        if counter == 0:
            print("\t\tElement", counter + 1, "hotov.\t\t[", counter + 1, "/", length, "]")
        elif counter % 100 == 0:
            print("\t\tElement", counter, "hotov.\t\t[", counter, "/", length, "]")
        elif counter + 1 == length:
            print("\t\tElement", counter + 1, "hotov.\t\t[", counter + 1, "/", length, "]")

        if False:
            # Vykreslení bodů daného elemenetu
            # Vytvoření seznamu indexů odpovídajících klíčových bodů
            # matching_indices = [m.queryIdx for m in good_matches]

            # Vytvoření seznamu souřadnic odpovídajících good_matches
            matched_keypoints1 = [selected_keypoints[m.queryIdx].pt for m in good_matches]
            matched_keypoints2 = [keypoints2[m.trainIdx].pt for m in good_matches]

            # Převod obrázku z BGR do RGB pro použití s Matplotlib
            image_rgb = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

            # Vykreslení bodů na obrázku
            for point in matched_keypoints1:
                plt.plot(point[0], point[1], 'ro', markersize=5)

            # Zobrazení obrázku s vykreslenými body
            plt.imshow(image_rgb)
            plt.axis('off')
            plt.tight_layout()
            plt.show()

            # Vykreslení shod na obrazu
            matched_image = cv2.drawMatches(image1, selected_keypoints, image2, keypoints2, good_matches, None,
                                            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

            # Zobrazení výsledku
            cv2.namedWindow('Matched image', cv2.WINDOW_NORMAL)
            cv2.resizeWindow('Matched image', int(0.25 * width), int(0.25 * height))
            cv2.imshow("Matched image", matched_image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        counter += 1

    print("\n\tHledání elementů dokončeno.")

    outcome = np.zeros((len(x_old), 4))
    outcome[:, 0], outcome[:, 1], outcome[:, 2], outcome[:, 3] = x_old, y_old, x_new, y_new
    # outcome[:, 4], outcome[:, 5] = angle_old, angle_new

    return outcome, index

In [ ]:
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from matplotlib.patches import Polygon as mpltPolygon


def plot_heatmap(big_polygon_coords, subregions_coords, subregion_values):
    fig, ax = plt.subplots()

    # Vykreslení velké oblasti
    big_polygon_patch = mpltPolygon(big_polygon_coords, closed=True, fill=None, color='royalblue')
    ax.add_patch(big_polygon_patch)

    # Vykreslení podoblastí a jejich hodnot jako heatmapy
    for i, subregion_coords in enumerate(subregions_coords):
        subregion_value = subregion_values[i]

        # Normalizace hodnoty pro lepší zobrazení barev na heatmapě
        normalized_value = (subregion_value - min(subregion_values)) / (max(subregion_values) - min(subregion_values))

        # Vybrání vhodné barvy pro heatmapu
        heatmap_color = plt.cm.jet(normalized_value)

        subregion_patch = mpltPolygon(subregions_coords[i], closed=True, alpha=1, facecolor=heatmap_color)
        ax.add_patch(subregion_patch)

    plt.axis('equal')

    # Přidání stupnice barev (colorbar)
    scalar_map = plt.cm.ScalarMappable(cmap='jet')
    scalar_map.set_array(subregion_values)
    plt.colorbar(scalar_map)

    plt.show()


# Příklad použití
big_polygon_coords = [[0, 0], [0, 10], [10, 10], [10, 0]]
subregions_coords = [
    [[0, 0], [0, 5], [5, 0]],
    [[0, 5], [5, 0], [5, 5]],
    [[0, 5], [5, 5], [5, 10]],
    [[0, 5], [10, 5], [5, 10]],
    [[5, 0], [10, 0], [10, 5]],
    [[5, 0], [5, 5], [10, 5]],
    [[0, 5], [0, 10], [5, 10]],
    [[10, 5], [10, 10], [5, 10]],
]
subregion_values = [0, 5, 3.56, 8.45, 1.3, 10, 9, 1.5, 10, 7, 6.12]

plot_heatmap(big_polygon_coords, subregions_coords, subregion_values)
